# Artificial Intelligence - Fall 2020 - Laboratory 10

## _First Order Predicate Logic:   Semantic Networks_

c: Alexandra Dobrescu <alexandramaria.digital@gmail.com>

## Introduction

The purpose of this laboratory is to understand the concepts related to semantic networks and to implement simple mechanisms for determining the attributes of objects in a semantic network.

### Classes, attributes, objects

We consider that any **object** can belong to a **class**. This association is described by the relation $ Object \xrightarrow{isa} Class $. We will represent this relation by an instance of the predicate $ isa $, in the form $ isa (Object, Class) $.

One class can **inherit** another class (it is a sub-type). This relationship is called A-Kind-Of and is described as $ Subclass \xrightarrow{ako} Class $. We will represent this relation by an instance of the predicate $ ako $, in the form $ ako (Subclass, Class) $.

A class can have **attributes** that have **values**. We represent that a class $ Class $ has the attribute $ Atr $ with the value $ val $ as an atom $ Atr (Class, value) $. The same will be for an object.

Attributes are inherited from a class to a subclass of it and from the class to the object (instance). To determine what the value of an attribute might be for a particular object, we need to go up (to superclasses) on the $ isa $ and $ ako $ relations to find the first (closest) definition of the value of that attribute, in an ancestor of the object class.

We will represent a semantic network as a knowledge base containing exclusively atoms. The predicates will be $ isa $, $ ako $, and the predicates corresponding to the various attributes.

## Useful functions from previous laboratories

### Task 0

Save the solution of `lab 8` (*Representation and Unification*) with the name `Lab08.py`. We will also use the functions already implemented in `Homework3_Resolution`.

In [1]:
from Lab08 import make_var, make_const, make_atom, make_or, make_neg, \
                is_variable, is_constant, is_atom, is_function_call, \
                get_args, get_head, get_name, get_value
from Homework3_Resolution import add_statement, print_KB, print_formula
from LPTester import testA, testFKB, testL, test_batch, testBatch

## Knowledge base

We have the following statements:

1. Most cars, but not all, are powered by an internal combustion engine (ICE).
2. An ICE provides a typical car with about 110 HP.
3. A hybrid car is a car that has an ICE and an electric motor.
4. There are two categorizations for hybrid cars:
5. you can have parallel versus series hybrids;
6. and you can have mild versus full hybrids.
7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor.
8. Some cars can be assisted by a low power electric motor (about 20HP).
9. A mild hybrid is a hybrid that has an ICE assisted by such a low power electric motor.
10. An electric car is considered to be able to be powered by electric power alone.
11. Electric cars have about 120HP, and no ICE.
12. A full hybrid can be powered only by its electric motor, so it could be considered a kind of electric car.
13. Full hybrids feature electric motors of about 100HP.
14. The Toyota Prius is a full and parallel hybrid.
15. The Honda Insight is a mild parallel hybrid.
16. The Chevrolet Volt is a full, series hybrid.
17. The Nissan Leaf is an electric car.

### Task 1

In the description of the knowledge base below, add the logical sentences corresponding to the indicated statements.

In [49]:
hybrids_KB = []
# 1. Most cars, but not all, are powered by an internal combustion engine (ICE).
# 2. An ICE provides a typical car with about 110 HP.
add_statement(hybrids_KB, make_atom("gas-power", make_const("car"), make_const("110")))
# 3. A hybrid car is a car that has an ICE and an electric motor.
add_statement(hybrids_KB, make_atom("ako", make_const("hybrid-car"), make_const("car")))
# 4. There are two categorizations for hybrid cars:
# 5. you can have parallel versus series hybrids (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("parallel-hybrid"), make_const("hybrid-car")))
# 5. you can have parallel versus series hybrids (part 2)
add_statement(hybrids_KB, make_atom("ako", make_const("series-hybrid"), make_const("hybrid-car")))
# 6. and you can have mild versus full hybrids. (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("mild-hybrid"), make_const("hybrid-car")))
# 6. and you can have mild versus full hybrids. (part 2)
add_statement(hybrids_KB, make_atom("ako", make_const("full-hybrid"), make_const("hybrid-car")))
# 7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor. (part 1)
add_statement(hybrids_KB, make_atom("electric-power", make_const("hybrid-car"), make_const("30")))
# 7. A hybrid would typically provide 100HP from the ICE and 30HP from the electric motor. (part 2)
add_statement(hybrids_KB, make_atom("gas-power", make_const("hybrid-car"), make_const("100")))
# 8. Some cars can be assisted by a low power electric motor (about 20HP) (part 1)
add_statement(hybrids_KB, make_atom("ako", make_const("assisted-electric"), make_const("car")))
# 8. Some cars can be assisted by a low power electric motor (about 20HP) (part 2)
add_statement(hybrids_KB, make_atom("electric-power", make_const("assisted-electric"), make_const("20")))
# 9. A mild hybrid is a hybrid that has an ICE assisted by such a low power electric motor.
add_statement(hybrids_KB, make_atom("ako", make_const("mild-hybrid"), make_const("assisted-electric")))
# 10. An electric car is a car powered by electric power alone.
# TODO
add_statement(hybrids_KB, make_atom("ako", make_const("electric-car"), make_const("car")))
# 11. Electric cars have about 120HP, and no ICE. (part 1)
# TODO
add_statement(hybrids_KB, make_atom("electric-power", make_const("electric-car"), make_const("120")))
# 11. Electric cars have about 120HP, and no ICE. (part 2)
# TODO
add_statement(hybrids_KB, make_atom("gas-power", make_const("electric-car"), make_const("0")))
# 12. A full hybrid can be powered only by its electric motor, so it could be considered a kind of electric car.
# TODO
add_statement(hybrids_KB, make_atom("ako", make_const("full-hybrid"), make_const("electric-car")))
# 13. Full hybrids feature electric motors of about 100HP.
# TODO
add_statement(hybrids_KB, make_atom("electric-power", make_const("full-hybrid"), make_const("100")))
# 14. The Toyota Prius is a full and parallel hybrid.
add_statement(hybrids_KB, make_atom("isa", make_const("toyota-prius"), make_const("parallel-hybrid")))
# 14. The Toyota Prius is a full and parallel hybrid.
add_statement(hybrids_KB, make_atom("isa", make_const("toyota-prius"), make_const("full-hybrid")))
# 15. The Honda Insight is a mild parallel hybrid.
# TODO
add_statement(hybrids_KB, make_atom("isa", make_const("honda-insight"), make_const("mild-hybrid")))
# 15. The Honda Insight is a mild parallel hybrid.
# TODO
add_statement(hybrids_KB, make_atom("isa", make_const("honda-insight"), make_const("parallel-hybrid")))
# 16. The Chevrolet Volt is a full, series hybrid.
# TODO
add_statement(hybrids_KB, make_atom("isa", make_const("chevrolet-volt"), make_const("full-hybrid")))
# 16. The Chevrolet Volt is a full, series hybrid.
# TODO
add_statement(hybrids_KB, make_atom("isa", make_const("chevrolet-volt"), make_const("series-hybrid")))
# 17. The Nissan Leaf is an electric car.
# TODO
add_statement(hybrids_KB, make_atom("isa", make_const("nissan-leaf"), make_const("electric-car")))

print("This is how the knowledge base is presented:")
print_KB(hybrids_KB)
print("==================== \n Inside the KB we can notice:")
print("\n".join([str(s) for s in hybrids_KB]))

OK: Added statement gas-power(car, 110)
OK: Added statement ako(hybrid-car, car)
OK: Added statement ako(parallel-hybrid, hybrid-car)
OK: Added statement ako(series-hybrid, hybrid-car)
OK: Added statement ako(mild-hybrid, hybrid-car)
OK: Added statement ako(full-hybrid, hybrid-car)
OK: Added statement electric-power(hybrid-car, 30)
OK: Added statement gas-power(hybrid-car, 100)
OK: Added statement ako(assisted-electric, car)
OK: Added statement electric-power(assisted-electric, 20)
OK: Added statement ako(mild-hybrid, assisted-electric)
OK: Added statement ako(electric-car, car)
OK: Added statement electric-power(electric-car, 120)
OK: Added statement gas-power(electric-car, 0)
OK: Added statement ako(full-hybrid, electric-car)
OK: Added statement electric-power(full-hybrid, 100)
OK: Added statement isa(toyota-prius, parallel-hybrid)
OK: Added statement isa(toyota-prius, full-hybrid)
OK: Added statement isa(honda-insight, mild-hybrid)
OK: Added statement isa(honda-insight, parallel-hyb

### Task 2

Implement the 3 helper functions below. Their functionality refers to the parents or attributes declared **explicitly** for a certain node (no network traversal is required).

In [50]:
# The function receives the name of a concept from the semantic network (object or class) and the name of an attribute
# and returns the value of the attribute for that concept, if given _explicitly_.
def get_attribute(node_name, attribute_name, net):
    # TODO
    for statement in net:
        #print(statement)
        args = get_args(statement)
        #print(args)
        #print(get_head(statement))
        #print(args[0][1])
        if get_head(statement) == attribute_name and args[0][1] == node_name:
            return args[1][1]
    pass

# The function returns a list of the names of the classes to which the object with the given name belongs
def make_isa_ancestor_list(node_name, net):
    # TODO
    classes = []
    for statement in net:
        args = get_args(statement)
        if get_head(statement) == "isa" and args[0][1] == node_name:
            #print(get_head(statement))
            #print(args[0][1])
            #print(args[1][1])
            classes.append(args[1][1])
    #print(classes)
    if len(classes)>0:
        return classes
    pass

# The function returns a list of class names that the class inherits with the given name
def make_ako_ancestor_list(node_name, net):
    # TODO
    superclasses = []
    for statement in net:
        args = get_args(statement)
        if get_head(statement) == "ako" and args[0][1] == node_name:
            #print(get_head(statement))
            #print(args[0][1])
            #print(args[1][1])
            superclasses.append(args[1][1])
    #print(classes)
    if len(superclasses)>0:
        return superclasses
    pass

# Test!
testBatch['sem-net-1'] = [(testFKB(testA, get_attribute, hybrids_KB), [
    # 0
    (('car', 'gas-power'), '110'),
    (('hybrid-car', 'electric-power'), '30'),
    (('hybrid-car', 'gas-power'), '100'),
    (('electric-car', 'electric-power'), '120'),
    (('electric-car', 'gas-power'), '0'),
        ]),
                      (testFKB(testL, make_isa_ancestor_list, hybrids_KB, True), [
    # 5
    ('toyota-prius', ['full-hybrid', 'parallel-hybrid']),
    ('chevrolet-volt', ['full-hybrid', 'series-hybrid']),
    ('nissan-leaf', ['electric-car']),
            ]),
                      (testFKB(testL, make_ako_ancestor_list, hybrids_KB, True), [
    # 8
    ('hybrid-car', ['car']),
    ('mild-hybrid', ['hybrid-car', 'assisted-electric']),
    ('full-hybrid', ['hybrid-car', 'electric-car']),
        ])]

test_batch('sem-net-1')

>>> Test batch [sem-net-1]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
>>>  11 / 11 tests successful.


### Task 3

Implement the `infer_attr` function that determines the value inherited by a node for an attribute. Get one of the values that can be inherited.

BONUS: get the closest value that can be inherited.

In [81]:
# infer_attr finds the value of the given name attribute, inferred for the given name node. It is enough to find a value
# in any ancestor of the node (considering the condition that there is no other definition on the path between 
# the node and the ancestors of the attribute).
# It will return a tuple consisting of the value and the name of the node where the value was found
# Returns None if no value can be found
# BONUS: implement a function which returns the closest value to the given node in the semantic network.
# If two different values are found, at a distance equal to the node, the message "CONTRADICTION" is returned instead.

def infer_attr(node_name, attribute_name, net):
    # TODO
    # L stores tuples formed of a discovered ancestor and the inferential distance to it
    L = []
    L.append((node_name,0))
    isa = make_isa_ancestor_list(node_name, net)
    if isa:
        for node in isa:
            L.append((node,1))
    CAND = []
    while L != []:
        (N,d) = L[0]
        L.remove((N,d))
        if get_attribute(N, attribute_name, net):
            CAND.append((N,d))
        else:
            ako = make_ako_ancestor_list(N, net)
            if ako:
                for node in make_ako_ancestor_list(N, net):
                    L.append((node,d+1))
    #print(CAND)
    
    for i in range(len(CAND)):
        if i<len(CAND):
            (C,d) = CAND[i]
            for j in range(len(CAND)):
                if j<len(CAND) and i!=j:
                    (C_prime, d_prime) = CAND[j]
                    #print(d_prime)
                    #print(d)
                    if d_prime < d or C_prime == C:
                        CAND.remove((C,d))
                        #print(CAND)
    
    if len(CAND) == 0:
        return None
    if len(CAND) == 1:
        (C1, d) = CAND[0]
        return (get_attribute(C1, attribute_name, net), C1)
    if len(CAND) > 1:
        return "CONTRADICTION"
    
    pass

# Test!
testBatch['sem-net-2'] = [(testFKB(testA, infer_attr, hybrids_KB), [
    #0
    (('car', 'gas-power'), ('110', 'car')),
    (('car', 'electric-power'), None),
    (('series-hybrid', 'gas-power'), ('100', 'hybrid-car')),
    #3
    (('nissan-leaf', 'electric-power'), ('120', 'electric-car')),
    (('nissan-leaf', 'gas-power'), ('0', 'electric-car')),
    (('full-hybrid', 'electric-power'), ('100', 'full-hybrid')),
    # BONUS:
    #6
    (('chevrolet-volt', 'electric-power'), ('100', 'full-hybrid')),
    (('mild-hybrid', 'gas-power'), ('100', 'hybrid-car')),
    (('mild-hybrid', 'electric-power'), 'CONTRADICTION'),
    (('honda-insight', 'electric-power'), 'CONTRADICTION'),
    (('honda-insight', 'gas-power'), ('100', 'hybrid-car')),
            ])]
test_batch('sem-net-2')

>>> Test batch [sem-net-2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
>>>  11 / 11 tests successful.


# Feedback time!

Hi!

I know I wasn't really vocal about my opinion during the discussion today. That's because I feel quite impartial to be honest. I just don't want you to take away from this that we don't like you or that we aren't interested in this class. We all want to learn and I think people's frustrations built up over not being able to solve things on their own or with some help from you and then using web resources and getting a 0, as well as being afraid that getting bad grades on labs or homeworks in this class could mean not being allowed to take the exam and, consequently, not getting their diploma this year. No one dislikes you, they're just afraid of failing!

I have no doubt that you want us to do great in this subject just as much as we do. That's why I think it's worth hearing out your students when they tell you what they need (in this case, more guidance during class, even when no specific questions are asked). 

Anyways, I really look forward to the ML labs, also because I will be using ML in my diploma project :D

See you in January. Have a nice winter break!

Irina/Mona